In [1]:
import pandas as pd
import yaml
import argparse
import torch
from model.model_viz import GoalNet

In [2]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [3]:
CONFIG_FILE_PATH = 'config/pie.yaml'  # yaml config file containing all the hyperparameters
DATASET_NAME = 'pie'

TEST_DATA_PATH = 'data/PIE/test_pie.pkl'
TEST_IMAGE_PATH = 'data/PIE/test'  # only needed for YNet, PECNet ignores this value
OBS_LEN = 15  # in timesteps
PRED_LEN = 45  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

ROUNDS = 1  # How often to evaluate the whole dataset, get stochastic mean results
BATCH_SIZE = 1

#### Load config file and print hyperparameters

In [4]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'resize': 0.25,
 'batch_size': 8,
 'train_mode': 'scene',
 'matrics': 'mse',
 'learning_rate': 0.0001,
 'num_epochs': 50,
 'grad_accum': True,
 'batch_size_gpu': 1,
 'data_augmentation': ['rot', 'flip'],
 'encoder_channels': [32, 32, 64, 64, 64],
 'decoder_channels': [64, 64, 64, 32, 32],
 'waypoints': [44],
 'feature_extract': True,
 'segmentation_model': None,
 'bbox_wh_model': 'pretrained_models/bbox_wh/pie_wh-72.pth',
 'feature_classes': 6,
 'temperature': 1.0,
 'loss_scale': 1000,
 'kernlen': 31,
 'nsig': 4,
 'clip_grad': False,
 'max_norm': 20,
 'use_TTST': True,
 'rel_threshold': 0.0001,
 'use_CWS': False,
 'CWS_params': {'sigma_factor': 6, 'ratio': 2, 'rot': True}}

#### Load preprocessed Data

In [5]:
df_test = pd.read_pickle(TEST_DATA_PATH)

In [6]:
df_test.head()

,frame,trackId,x,y,w,h,sceneId,metaId
0,01478,0,1124.760,742.35,7.92,27.5,set03-video_0001-01478,0
1,01478,0,1124.760,742.35,7.92,27.5,set03-video_0001-01478,0
2,01478,0,1125.235,743.20,8.01,27.7,set03-video_0001-01478,0
3,01478,0,1125.710,744.04,8.08,27.9,set03-video_0001-01478,0
4,01478,0,1126.185,744.89,8.17,28.1,set03-video_0001-01478,0


#### Initiate model and load pretrained weights

In [7]:
model = GoalNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

In [8]:
model.load('pretrained_models/pie_adam_final_7_weights.pt')

<All keys matched successfully>


#### Evaluate model

In [9]:
model.evaluate(df_test.loc[df_test['sceneId'] == 'set03-video_0012-05669'][:60], params, image_path=TEST_IMAGE_PATH, batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1003.42it/s]


Start testing


Round: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.31s/it]

skip
Round 0: 
Test CMSE: 9.190201759338379 
Test CFMSE: 34.420894622802734
Test MSE(0.5): 4.329476356506348 
Test MSE(1.0): 7.7779107093811035 
Test MSE(1.5): 13.949362754821777


Average performance over 1 rounds: 
Test CMSE: 9.190201759338379 
Test CFMSE: 34.420894622802734 
Test MSE(0.5): 4.329476356506348 
Test MSE(1.0): 7.7779107093811035 
Test MSE(1.5): 13.949362754821777


In [10]:
model.evaluate(df_test.loc[df_test['sceneId'] == 'set03-video_0001-17086'][:60], params, image_path=TEST_IMAGE_PATH, batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1003.42it/s]


Start testing


Round: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.35it/s]